In [57]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

import torchvision
from torchvision import models, datasets, transforms

import ast

In [93]:
EPOCH = 5
BATCH_SIZE = 10
LR = 1e-4

In [95]:
df = pd.read_csv("new_dataset.csv")
data_target = np.asarray(df[["data", "target"]])
np.random.shuffle(data_target)

train_X = data_target[:75, 0]
train_y = data_target[:75, 1]

valid_X = data_target[75:, 0]
valid_y = data_target[75:, 1]

for i in range(train_X.shape[0]):
    train_X[i] = np.asarray(ast.literal_eval(train_X[i]))

for i in range(valid_X.shape[0]):
    valid_X[i] = np.asarray(ast.literal_eval(valid_X[i]))

train_X = np.array([x for x in train_X])
train_y = np.asarray(train_y, dtype=np.int16)

valid_X = np.array([x for x in valid_X])
valid_y = np.asarray(valid_y, dtype=np.int16)

train_X = torch.from_numpy(train_X).float()
train_y = torch.from_numpy(train_y).long()
valid_X = torch.from_numpy(valid_X).float()
valid_y = torch.from_numpy(valid_y).long()

train_X

tensor([[[279.0000, 217.0000,   9.3409],
         [279.0000, 217.0000,   9.3409],
         [279.0000, 217.0000,   9.3409],
         ...,
         [279.0000, 217.0000,   9.3409],
         [282.0000, 169.0000,   9.4869],
         [282.0000, 169.0000,   9.4869]],

        [[359.0000, 319.0000,  37.6564],
         [274.0000, 189.0000,   8.8601],
         [274.0000, 189.0000,   8.8601],
         ...,
         [279.0000, 149.0000,   8.7322],
         [279.0000, 149.0000,   8.7322],
         [279.0000, 149.0000,   8.7322]],

        [[306.0000, 430.0000,  39.2427],
         [267.0000, 213.0000,   8.8601],
         [267.0000, 213.0000,   8.8601],
         ...,
         [268.0000, 193.0000,   8.8601],
         [268.0000, 193.0000,   8.8601],
         [268.0000, 193.0000,   8.8601]],

        ...,

        [[396.0000,  22.0000,  11.9270],
         [291.0000, 209.0000,   7.9058],
         [291.0000, 209.0000,   7.9058],
         ...,
         [293.0000, 189.0000,   8.0778],
         [293.0000, 18

In [96]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()

        self.x = x
        self.y = y

    def __getitem__(self, index):
        
        x = torch.Tensor(self.x[index])
        y = torch.Tensor(self.y[index])

        return (x, y)
    
    def __len__(self):
        count = self.x.shape[0]
        return count

train_dataset = CustomDataset(train_X, train_y)
valid_dataset = CustomDataset(valid_X, valid_y)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)


In [5]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.dropout = 0.5

        self.encoder = nn.Sequential(
            nn.Linear(10, 4098),
            nn.ReLU(),
            nn.Dropout(p=self.dropout),

            nn.Linear(4098, 16392),
            nn.ReLU(),
            nn.Dropout(p=self.dropout),

            nn.Linear(16392, 50176),
        )

        self.decoder = nn.Sequential(
            nn.Linear(50176, 16392),
            nn.ReLU(),
            nn.Dropout(p=self.dropout),

            nn.Linear(16392, 4098),
            nn.ReLU(),
            nn.Dropout(p=self.dropout),

            nn.Linear(4098, 10),
        )

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)

        x = self.decoder(x)
        x = x.transpose(1, 2)

        return x

    def encode(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)

        return x

    def decode(self, x):
        x = self.decoder(x)
        x = x.transpose(1, 2)

        return x    

In [72]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
auto_encoder = AutoEncoder().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(auto_encoder.parameters(), lr=LR, weight_decay=1e-4)

for epoch in range(EPOCH):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0

    auto_encoder.train()
    for i, datum in enumerate(train_dataloader):
        inputs, labels = datum[0].to(device), datum[0].to(device).clone().detach()

        optimizer.zero_grad()
        outputs = auto_encoder(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    auto_encoder.eval()
    with torch.no_grad():
        for i, datum in enumerate(valid_dataloader):
            inputs, labels = datum[0].to(device), datum[0].to(device).clone().detach()

            outputs = auto_encoder(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)

    avg_train_loss = train_loss / len(train_dataloader.sampler)
    avg_valid_loss = valid_loss / len(valid_dataloader.sampler)
    
    print(f"=== Epoch {epoch} ===")
    print(f"Average Training Loss: {avg_train_loss}")
    print(f"Average Validation Loss: {avg_valid_loss}")

NameError: name 'AutoEncoder' is not defined

In [67]:
torch.cuda.empty_cache()

In [97]:
class Modified_ResNet(nn.Module):
    def __init__(self):
        super(Modified_ResNet, self).__init__()
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 101)

        for param in self.resnet.parameters():
            param.requires_grad = False

        # Unfreeze the parameters of the last few layers for fine-tuning
        for param in self.resnet.layer4.parameters():
            param.requires_grad = True

        
        self.in_fc = nn.Linear(10, 50176)

    def forward(self, x):
        x = x.transpose(1, 2).to(x.device)
        x = self.in_fc(x)
        
        x = x.view(-1, 3 * 224 * 224)
        
        min_val = torch.min(x, dim=1)[0].view(-1, 1)
        max_val = torch.max(x, dim=1)[0].view(-1, 1)
        x = (x - min_val) / (max_val - min_val)
        
        x = x.view(-1, 3, 224, 224).to(x.device)
        x = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(x)

        x = self.resnet(x)
        
        return x

In [98]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Modified_ResNet().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)

for epoch in range(EPOCH):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0

    model.train()
    for i, datum in enumerate(train_dataloader):
        inputs, labels = datum[0].to(device), datum[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    model.eval()
    with torch.no_grad():
        for i, datum in enumerate(valid_dataloader):
            inputs, labels = datum[0].to(device), datum[1].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            
            predicted = torch.softmax(outputs, dim=1)
            predicted = torch.argmax(predicted, dim=1)

            correct += (predicted == labels).sum().item()

    avg_train_loss = train_loss / len(train_dataloader.sampler)
    avg_valid_loss = valid_loss / len(valid_dataloader.sampler)
    
    print(f"=== Epoch {epoch} ===")
    print(f"Average Training Loss: {avg_train_loss}")
    print(f"Average Validation Loss: {avg_valid_loss}")
    print(f"Validation Accuracy: {correct / len(valid_dataloader.sampler)}")


=== Epoch 0 ===
Average Training Loss: 4.846560509999593
Average Validation Loss: 4.951395511627197
Validation Accuracy: 0.0
=== Epoch 1 ===
Average Training Loss: 3.856315024693807
Average Validation Loss: 4.8973774909973145
Validation Accuracy: 0.0
=== Epoch 2 ===
Average Training Loss: 3.2841781616210937
Average Validation Loss: 4.9279398918151855
Validation Accuracy: 0.0
=== Epoch 3 ===
Average Training Loss: 2.8388689359029136
Average Validation Loss: 4.9548234939575195
Validation Accuracy: 0.0
=== Epoch 4 ===
Average Training Loss: 2.5239983558654786
Average Validation Loss: 4.889196395874023
Validation Accuracy: 0.0


In [99]:
model.eval()

# data = torch.tensor([[279, 231, 8.381627082824707], [279, 231, 8.381627082824707], [279, 231, 8.381627082824707], [279, 231, 8.381627082824707], [279, 231, 8.381627082824707], [279, 180, 9.34087085723877], [279, 180, 9.34087085723877], [279, 158, 9.486932754516602], [279, 158, 9.486932754516602], [279, 158, 9.486932754516602]], dtype=torch.float32)
# data = nn.functional.normalize(data)

# data = data.view(1, 10, 3).to(device)
# output = model(data)

# output = torch.softmax(output, dim=1)
# output = torch.argmax(output, dim=1)
# print(output.item())

for datum in valid_dataloader:
    inputs, labels = datum[0].to(device), datum[1].to(device)
    
    outputs = model(inputs)
    predicted = torch.softmax(outputs, dim=1)
    predicted = torch.argmax(predicted, dim=1)

    print(predicted)

tensor([38, 80, 85, 38, 38, 85, 38, 38, 53, 38], device='cuda:0')


In [100]:
torch.save(model, "Modified_ResNet.pth")